# **Spotting Fake News with Python**

---
## **Enviroment Setup and Loading Data**

---
## **Exploring the news and their content**

---
## **Applying machine learning to detect fake news**

---
## **Experiment Results and Comments**